In [1]:
from webbot import Browser
from selenium import webdriver
import time
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import csv
from datetime import datetime

## ETAPE 1 : SE CONNECTER A ATOM

In [2]:
def access_atom(driver):
    """
    Accède à la plateforme Phoebus en utilisant un objet de navigateur.

    Cette fonction effectue les étapes suivantes :
    1. Navigue vers la page d'accueil de Atom Archives.
    2. Clique sur le bouton "Ouverture de session".
    3. Utilise les identifiants stockés dans le fichier 'credentials.json' pour se connecter.

    Args:
        driver: Objet de navigateur permettant d'automatiser les interactions avec la page web.
    Note:
        Assurez-vous d'avoir un fichier 'credentials.json' contenant les champs 'email' et 'password'
        pour effectuer la connexion à Phoebus.
    """
        
    driver.go_to('https://atom-archives.unil.ch/') ##METTRE LA BONNE ADRESSE URL ∆ PAS LE MODE STAGE
    driver.click('Ouverture de session', tag='button')
    
    with open('credentials.json', 'r') as jsonFile:
        credentials = json.load(jsonFile)
        
    driver.type(credentials['email'], id='email')
    driver.type(credentials['password'], id='password')
    driver.click('Ouverture de session')

# LANCER LA FONCTION
driver = Browser() #Browser pour ouvrir un navigateur
access_atom(driver)

## ETAPE 2 : ACQUISITION DES DONNEES

In [3]:
def extract_autorite(csv_path):
    """
    Extrait les noms à partir du fichier CSV contenant les noms et les données récupérées.

    Cette fonction charge un fichier CSV, remplace les valeurs NaN par une chaîne vide,
    et remplace les NaN dans la colonne 'DateExistence' par "n.c.". Ensuite, elle extrait
    les autorités de la colonne 'autoritesPHOEBUS' du DataFrame.

    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données récupérées.

    Returns:
        autorities (list): Liste des données récupérées.
    """
    
    # Charger le fichier CSV
    csv_path = '01_noticesACompleter.csv'
    df = pd.read_csv(csv_path, na_values=['NULL'], delimiter=',') #delimiter dépend de la configuration du separator du csv
    
    # Extraire les noms de la colonne "url_Phoebus"
    autorities = df.values.tolist()

    return autorities
    
# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorities = extract_autorite(csv_path)
# Afficher les noms (à titre de vérification)
print(autorities)

[['https://atom-archives.unil.ch/index.php/rossier-edmond', 'ROSSIER Edmond', '1865-1945', nan, 'Historien et journaliste.', nan, 'http://www.idref.fr/073313653/id.', nan, 'idref_073313653'], ['https://atom-archives.unil.ch/index.php/trolliet-gilbert', 'TROLLIET Gilbert', '1907-1980', 'poète', nan, nan, 'http://www.idref.fr/027169952/id.', nan, 'idref_027169952'], ['https://atom-archives.unil.ch/index.php/clerc-jean-louis', 'CLERC Jean-Louis', '1908-1961', 'artiste et écrivain', nan, nan, 'http://www.idref.fr/080797741/id.', nan, 'idref_080797741'], ['https://atom-archives.unil.ch/index.php/cornut-samuel', 'CORNUT  Samuel', '1861-1918', nan, nan, nan, 'http://www.idref.fr/198600275/id.', nan, 'idref_198600275'], ['https://atom-archives.unil.ch/index.php/cuneo-anne', 'CUNEO Anne', '1936-2015', nan, 'Auteure dramatique, essayiste, metteure en scène de théâtre, cinéaste. ', nan, 'http://www.idref.fr/026807114/id.', nan, 'idref_026807114'], ['https://atom-archives.unil.ch/index.php/delacos

## ETAPE 3 : ENTRER DANS LA NOTICE EN MODE EDITION

In [4]:
def open_autorites(csv_path):
    """
    Charge un fichier CSV, ajoute le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "notice 1",
    puis extrait les noms modifiés de la colonne "notice 1".

    Args:
        csv_path (str): Chemin du fichier CSV.
    Returns:
        list: Liste des URLs renvoyant à l'édition des notices d'autorité, ouverte sur la Zone d'identification.
    """

    # Charger le fichier CSV
    df = pd.read_csv(csv_path, delimiter=',') #delimiter dépend de la configuration du separator du csv

    # Extraire les noms de la colonne "autoritesPHOEBUS" et ajouter le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "notice 1" 
    df['url_phoebus'] = df['url_phoebus'].astype(str) + '/edit#identityArea'

    # Extraire les noms de la colonne "notice 1" modifiée
    autorites_url = df['url_phoebus'].tolist()
    
    return autorites_url

# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorites_url = open_autorites(csv_path)
print(autorites_url)

['https://atom-archives.unil.ch/index.php/rossier-edmond/edit#identityArea', 'https://atom-archives.unil.ch/index.php/trolliet-gilbert/edit#identityArea', 'https://atom-archives.unil.ch/index.php/clerc-jean-louis/edit#identityArea', 'https://atom-archives.unil.ch/index.php/cornut-samuel/edit#identityArea', 'https://atom-archives.unil.ch/index.php/cuneo-anne/edit#identityArea', 'https://atom-archives.unil.ch/index.php/delacoste-suzanne/edit#identityArea', 'https://atom-archives.unil.ch/index.php/grosjean-raoul/edit#identityArea', 'https://atom-archives.unil.ch/index.php/gregoire-helene/edit#identityArea', 'https://atom-archives.unil.ch/index.php/hersperger-charles/edit#identityArea', 'https://atom-archives.unil.ch/index.php/erni-barbara/edit#identityArea', 'https://atom-archives.unil.ch/index.php/peiry-alexis/edit#identityArea', 'https://atom-archives.unil.ch/index.php/poitry-guy/edit#identityArea', 'https://atom-archives.unil.ch/index.php/roche-sylviane/edit#identityArea', 'https://ato

## ETAPE 4 : OUVRIR LES ZONES ET SAISIR LES INFORMATIONS

In [5]:
def modify_autorites (driver, idRef_links, autorites_url):
    """
    Saisit dans les bons champs les informations récupérées dans IdRef.

    Cette fonction prend un objet 'driver' pour la navigation web automatisée, une liste d'autorités
    récupérées depuis IdRef, et une liste d'URLs pour les notices d'autorité. Elle ouvre chaque notice d'autorité
    avec le navigateur automatisé, ouvre toutes les Zones, saisit les informations appropriées dans les champs correspondants,
    et sauvegarde les modifications.

    Args:
        driver: Objet pour la navigation web automatisée.
        autorities (list): Liste d'autorités récupérées depuis IdRef.
        autorites_url (list): Liste d'URLs pour les notices d'autorité.

    Returns:
        None
    Notes :
        La fonction inclut les modifications automatiques effectuées par Botorité.
    """

    
    try:
        
        for i, autorite_url in enumerate(autorites_url) :
            
            driver.go_to(autorite_url)

            # Ouvrir toutes les Zones
            driver.click('Zone du contrôle')   
            driver.execute_script("window.scrollTo(0,0);")
            time.sleep(2)
            driver.click('Zone de description')
            # Récupérer le contenu HTML depuis la page
            content = driver.get_page_source()
            soup = BeautifulSoup(content, features='lxml')
            time.sleep(2)
            
            ## ZONE IDENTIFICATION
            # Modification du type d'entité
            driver.click(tag='select', id='entityType')
            driver.click('Personne', tag='option')

            
            ## ZONE DE CONTROLE
            # saisie de l'identifiant de notice d'autorité                      
            if driver.exists(id='descriptionIdentifier'):
                driver.type(autorities[i][8], id='descriptionIdentifier')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Identifiant de la description n'a pas été trouvée et saisie")
            
            
            # Modification de l'entretien du dépôt
            if driver.exists(id='maintainingRepository'):
                driver.type('Centre des littératures en Suisse romande', id='maintainingRepository')
                time.sleep(4)
                driver.press(driver.Key.ENTER)
            else:
                print(f"ERREUR pour {autorities[i][1]} : Entretien du dépôt de la description n'a pas été trouvé et saisi")
                   
            # Modification du statut
            driver.click(tag='select', id='descriptionStatus')
            driver.click('Final', tag='option')
            
            # saisie de l'historique de révision et de la référence
                # Obtenez la date et l'heure actuelles au format YYYY-MM-DD HH:MM:SS et formater le texte du champ Révision
            today_date = datetime.today().strftime('%Y-%m-%d à %H:%M:%S')
            revision_history_text = "Notice Phœbus. Dernière mise à jour : " + today_date
            
            if driver.exists(id='revisionHistory'):
                driver.type(revision_history_text, id='revisionHistory')
                print(revision_history_text)
            else:
                print(f"ERREUR pour {autorities[i][1]} : Dates de production et de révision n'ont pas été trouvées et saisies")
            
            if driver.exists(id='sources'):
                driver.type(f"Voir également l'Agence bibliographique de l'Enseignement supérieur (ABES) : {autorities[i][6]}", id='sources')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Sources n'ont pas été trouvées et saisies")
                
            # scroller vers le bas puis, sauvegarder la saisie
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)

            driver.click('Sauvegarder', tag='input')
            
            print(f"({i+1}) Opération de saisie terminée pour {autorities[i][1]} : {autorities[i][0]}")
        
    except Exception as e:
        print(f"Etape 9 : Erreur lors de la modification des notices d'autorité : {e}")
        
# LANCEMENT DE LA FONCTION
modify_autorites(driver, autorities, autorites_url)

Notice Phœbus. Dernière mise à jour : 2024-06-27 à 09:47:18
(1) Opération de saisie terminée pour ROSSIER Edmond : https://atom-archives.unil.ch/index.php/rossier-edmond
Etape 9 : Erreur lors de la modification des notices d'autorité : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.16.0 x86_64)



## LANCEMENT DES FONCTIONS

In [ ]:
# ETAPE 1
driver = Browser()
access_atom(driver)

# ETAPE 2
csv_path = '01_autorite_Id.csv'
autorities = extract_autorite(csv_path)
#print(autorities)

# ETAPE 3
csv_path = '01_autorite_Id.csv'
autorites_url = open_autorites(csv_path)

# ETAPE 4
modify_autorites(driver, idRef_links, autorites_url)